In [15]:
import scipy.stats as sta
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from numba import jit,vectorize
from ipyparallel import Client
import random
import numpy
from pyhmc import hmc
from sghmc.module import SGHMC, SGHMC_parallel

In [2]:
def SGHMC_friction(theta0, X, gradU, eps, sample_size, B, C, batch_size,burnin, M = None):
    '''
    SGHMC with friction. See details from Stochastic Gradient Hamiltonian Monte Carlo (Chen et al., 2014)
    
    INPUT:
    ----------------------------------------------------------------------
    theta0(1-dim numpy array): starting position of theta
    
    X(2-dims numpy array): data
    
    gradU(scale,theta,batch): function to compute gradient of U on a particular batch,
    
        INPUT:
        ----------------------------------------------------------------------
        scale: data size/ batch size
        theta: theta
        batch(numpy array): batch
        ----------------------------------------------------------------------
        OUTPUT:
        ----------------------------------------------------------------------
        thetat: Gradient of U on batch
        ----------------------------------------------------------------------
    
    eps: step size
    
    sample_size: number of samples drawn from prosterior distribution
    
    B: noise estimate
    
    C: user specified friction term
    
    batch_size: size of minibatches
    
    burnin: number of iterations for warm up
    
    M(optional): Mass matrix, defualt is set to be identity.
    ----------------------------------------------------------------------
    OUTPUT:
    ----------------------------------------------------------------------
    thetat: thetas sampled from posterior distribution
    ----------------------------------------------------------------------
    '''
    
    i = sample_size+burnin
    p = theta0.shape[0]
    thetat = np.zeros((i+1,p))
    thetat[0,:] = theta0
    
    m1 = int(np.ceil(X.shape[0]/batch_size))
    batch = [X[i::m1] for i in range(m1)]
    m = len(batch)

    if(M is None):
        M = np.eye(p)
    
    
    for t in range(i):
        ri = np.random.multivariate_normal(np.zeros(p), M)
        thetai = thetat[t]
        for j in range(m):
            thetai = thetai + eps * np.linalg.solve(M, ri)
            gU = gradU(X.shape[0]/batch[j].shape[0],thetai,batch[j])
            ri = ri - eps * gU - eps * C @ np.linalg.solve(M, ri) + np.random.multivariate_normal(np.zeros(p), np.array(2*eps*(C-B)).reshape(p,p))
        thetat[t+1] = thetai

    return thetat[burnin+1:]

## test basic code

In [3]:
def gradU(scale,theta,batch):
    
    def grad_log_data(x,theta):
        return -(-4*theta+4*theta**3)/50

    def grad_log_prior(theta):
        return 0
    return - scale*batch.shape[0]*grad_log_data(batch,theta)-grad_log_prior(theta)

In [4]:
@jit(nopython=True)
def gradU_numba(scale,theta,batch):
    return - scale*batch.shape[0]*(-(-4*theta+4*theta**3)/50)

In [5]:
%%timeit
SGHMC_friction(theta0 = np.array([0]),X = np.zeros((50,1)), gradU = gradU,
                eps=0.1,sample_size=10000, B=np.array([0]),C=2*np.eye(1),batch_size = 10,burnin=500)


11.7 s ± 87.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Profile Basic SGHMC function

In [6]:
profile = %prun -r -q SGHMC_friction(theta0 = np.array([0]),X = np.zeros((50,1)), gradU = gradU, eps=0.1,sample_size=2000, B=np.array([0]),C=2*np.eye(1),batch_size = 10,burnin=500)
profile.sort_stats('cumtime').print_stats(10)

          2132514 function calls (2057514 primitive calls) in 3.554 seconds

   Ordered by: cumulative time
   List reduced from 73 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.554    3.554 {built-in method builtins.exec}
        1    0.000    0.000    3.554    3.554 <string>:1(<module>)
        1    0.349    0.349    3.554    3.554 <ipython-input-2-86e513f1bfeb>:1(SGHMC_friction)
    15000    0.472    0.000    2.326    0.000 {method 'multivariate_normal' of 'numpy.random.mtrand.RandomState' objects}
145000/70000    0.156    0.000    1.707    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    15000    0.015    0.000    1.081    0.000 <__array_function__ internals>:2(allclose)
    15000    0.032    0.000    1.052    0.000 numeric.py:2103(allclose)
    15000    0.013    0.000    0.898    0.000 <__array_function__ internals>:2(isclose)
    15000    0.104    0.000    0.

## Use SqrtM @ Normal to replace multivariate normal

In [7]:
def SGHMC_friction2(theta0, X, gradU, eps, sample_size, B, C, batch_size,burnin, M = None):
    '''
    SGHMC with friction:
    theta0(numpy array): starting position of theta
    X: data
    grad_logden_data: gradient of the data log density
    grad_logden_prior: gradient of the prior log density
    eps: step size
    sample_size: number of samples drawn from prosterior distribution
    B: noise estimate
    C: user specified friction term
    batch_size: size of minibatches
    M: Mass matrix
    '''
    
    i = sample_size+burnin
    p = theta0.shape[0]
    thetat = np.zeros((i+1,p))
    thetat[0] = theta0
    
    m1 = int(np.ceil(X.shape[0]/batch_size))
    batch = [X[i::m1] for i in range(m1)]
    m = len(batch)

    if(M is None):
        M = np.eye(p)
    
    M1sqrt = np.linalg.cholesky(M)
    Msqrt = np.linalg.cholesky(2*eps*(C-B))
    
    for t in range(i):
        thetai = thetat[t]
        ri = M1sqrt@np.random.normal(size=p)
        for j in range(m):
            thetai = thetai + eps * np.linalg.solve(M,ri)
            gU = gradU(X.shape[0]/batch[j].shape[0],thetai,batch[j])
            ri = ri - eps * gU - eps * C @ np.linalg.solve(M, ri) + Msqrt@np.random.normal(size=p)
        thetat[t+1] = thetai
    
    return thetat[burnin+1:]

## test  on SGHMC2

In [8]:
%%timeit
SGHMC_friction2(theta0 = np.array([0]),X = np.zeros((50,1)), gradU = gradU, eps=0.1,sample_size=10000, B=np.array([0]),C=2*np.eye(1),batch_size = 10,burnin=500)

2.99 s ± 23.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Profile SGHMC2

In [9]:
profile = %prun -r -q SGHMC_friction2(theta0 = np.array([0]),X = np.zeros((50,1)), gradU = gradU, eps=0.1,sample_size=2000, B=np.array([0]),C=2*np.eye(1),batch_size = 10,burnin=500)
profile.sort_stats('cumtime').print_stats(10)

          802556 function calls in 0.982 seconds

   Ordered by: cumulative time
   List reduced from 33 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.982    0.982 {built-in method builtins.exec}
        1    0.000    0.000    0.982    0.982 <string>:1(<module>)
        1    0.222    0.222    0.982    0.982 <ipython-input-7-844daab9ca59>:1(SGHMC_friction2)
    25000    0.019    0.000    0.467    0.000 <__array_function__ internals>:2(solve)
    25002    0.016    0.000    0.444    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    25000    0.173    0.000    0.428    0.000 linalg.py:327(solve)
    12500    0.051    0.000    0.169    0.000 <ipython-input-3-876c899523f9>:1(gradU)
    15000    0.123    0.000    0.123    0.000 {method 'normal' of 'numpy.random.mtrand.RandomState' objects}
    12500    0.116    0.000    0.116    0.000 <ipython-input-3-876c899523f9>:3(grad_l

In [ ]:
%%timeit
theta1 = SGHMC_parallel(np.array([0]),np.zeros((50,1)),gradU_numba,0.1,10000, np.array([0]),2*np.eye(1),10,500)